PPD: OpenMP / Parallel Sections

Hélio - DC/UFSCar - 2023

**Parallel sections: dividindo trechos de código**

Usada também dentro de uma região paralela, a diretiva ***sections*** permite especificar blocos dentro dessa região que devem ser **divididos** entre as *threads* do time (não replicados).

Cada *section* é executada apenas uma vez, por **qualquer uma** das *threads* no time. Dependendo do número de seções e do número de *threads* no time, pode haver *threads* executando mais de uma seções, assim como é possível que alguma *thread* não tenha seção para executar.

Uma barreira é inserida automaticamente ao final das *sections*, exceto se a cláusula *nowait* for especificada na diretiva.

```
#pragma omg sections [clause ...]  newline
{
  [#pragma omp section newline]
    bloco_de_código
  [#pragma omp section newline]
    bloco_de_código
  ...
}
```
Cláusulas:
```
    private (list)
    first private (list)
    last private (list)
    reduction (operator: list)
    nowait
```
Vejamos o exemplo a seguir:

```
  #pragma omp parallel shared(a,b) private(i,tnum)
  {
    ... // Todas as threads do time executam esse trecho

    #pragma omp sections
    {
      #pragma omp section
      {
         faça_isso();
      }
      #pragma omp section
      {
         faça_aquilo();
      }
      #pragma omp section
      {
         faça_ainda_outra_coisa();
      }
    }   // Fim das seções

    ... // Todas as threads do time executam esse threcho

  }    // Fim da região paralela
}
```

Assim como com a divisão de trabalho usando *for*, a construção *sections* pode ser declarada junto com a região paralela, caso essa seja a única construção dentro do bloco de código:

```
#pragma omp parallel sections [clause[ [,] clause] ... ] new-line
{
  [#pragma omp section new-line]
    structured-block
  [#pragma omp section new-line
    structured-block]
   ...
}
```

In [ ]:
%%writefile ps.c 

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <omp.h>

#define N       10
#define NUM     3

void
func(int tnum)
{
  int i;
  for(i=0; i < NUM; i++) {
    printf("tnum: %d\n",tnum);
    sleep(rand()%5);
  }
}

int
main ()
{
  int i,tnum;
  float a[N], b[N];

  // Inicializacoes
  for (i=0; i < N; i++) {
    a[i] = i * 1.5;
    b[i] = i + 3.1415;
  }

  // Numero de secoes pode diferir do numero de threads / processadores disponiveis
  // Nesse caso, vale a politica de escalonamento para atribuicoes...

#pragma omp parallel shared(a,b) private(i,tnum)
{
  #pragma omp sections nowait
  {
    #pragma omp section
    {
      printf("First Section\n"); fflush(stdout);
      tnum=omp_get_thread_num();
      func(tnum);
    }
    #pragma omp section
    {
      tnum=omp_get_thread_num();
      printf("Section 2\n"); fflush(stdout);
      func(tnum);
    }       
    #pragma omp section
    {
      tnum=omp_get_thread_num();
      printf("3rd Section\n"); fflush(stdout);
      func(tnum);
    }
    #pragma omp section
    {
      tnum=omp_get_thread_num();
      printf("Section number 4\n"); fflush(stdout);
      func(tnum);
    }
  }  // fim das secoes 
}  // fim da regiao paralela 
        
  return(0);
}

Writing ps.c


In [ ]:
!gcc -Wall ps.c -o ps -fopenmp && ./ps
# ! OMP_NUM_THREADS=2 ./ps
# ! OMP_NUM_THREADS=4 ./ps
# ! lscpu

First Section
tnum: 0
Section 2
tnum: 3
3rd Section
tnum: 2
Section number 4
tnum: 1
tnum: 1
tnum: 3
tnum: 3
tnum: 2
tnum: 0
tnum: 1
tnum: 2
tnum: 0
